Link para melhor visualização do Notebook em seu ambiente de execução: [Notebook Colab](https://colab.research.google.com/drive/1qeqlEviy6-S9Ms99F6uJjIgUm7FVNGsn?usp=sharing)


---


Este notebook foi desenvolvido para construção dos arquivos utilizados pelo Semantic Knowledge and Interpretation Navigator for Nurturing Exact References (SKINNER) voltado para modelos WOKE. Esta estratégia de extração de referências visa realizar operações semelhantes às do treinamento de modelos Word2Vec para construção de Word Embeddings, agrupando tokens de contexto referentes aos seus respectivos tokens presentes no vocabulário dos modelos, analisando assim a formação das palavras de contexto para um determinado vetor de palavras. Esse agrupamento leva em consideração o vocabulário do modelo, o tamanho da janela de contexto utilizado no parâmetro de treinamento do modelo, e a frequência da co-ocorrencia de tais palavras no contexto para quantificar o grau de influência.

# Configuração de ambiente
Instalação e importação de bibliotecas que serão utilizadas ao decorrer das execuções.

Recomenda-se que sempre execute os programas usando o Google Colab, no qual os testes foram desenvolvidos.

In [ ]:
try:
  from google.colab import drive
  from google.colab import output
  drive.mount('/content/drive')
except Exception:
  GOOGLE_COLAB = False
else:
  GOOGLE_COLAB = True
finally:
  import os
  import msgpack
  import re
  from gensim.models import Word2Vec
  !pip install openpyxl
  import openpyxl
  !pip install gdown
  import gdown
  import platform

  OS_ATUAL = platform.system()
  CAMINHO_EXEC_ATUAL = os.getcwd()
  CAMINHO_SKINNER = os.path.join(CAMINHO_EXEC_ATUAL,'SKINNER_files')
  CAMINHO_PLANILHA_METADADOS = os.path.join(CAMINHO_SKINNER,'planilha_metadados_woke.xlsx')

Mounted at /content/drive


# Funções

In [ ]:
def organizarAmbienteExecucao():
  """
  Função responsável por organizar as pastas que serão utilizadas pelo SKINNER.
  É onde ocorre a criação da pasta "SKINNER_files" no diretório de execução.
  Esta pasta armazenará tanto a planilha de metadados, onde serão puxados os
  assuntos, links para a página do RI e para o PDF de cada trabalho, além dos
  arquivos de cada modelo analisado.
  """
  global CAMINHO_SKINNER
  try:
    os.makedirs(CAMINHO_SKINNER,exist_ok=True)
  except Exception:
    return False
  else:
    return True

def coletar_ID_link_Dive(link_drive : str) -> str:
  """
  Função responsável por coletar o ID de um arquivo do Google Drive com base no
  seu link de compartilhamento.

  ### Parâmetros:
  - link_drive: String contendo o link de compartilhamento do arquivo ou pasta
  no Google Drive.

  ### Retornos:
  - String contendo o ID do arquivo ou None, caso o link não esteja no padrão
  esperado.
  """
  padrao_regex = r'\/d\/(.+)/' # Padrão regex para identificar o link no meio do URL
  busca = re.search(padrao_regex,link_drive) # Buscando padrão dentro do link fornecido
  if busca: # Verificando se a busca pelo padrão dentro do link foi bem sucedida
    return busca.group(1)
  else:
    return None

def baixarArquivoDrive(ID_arquivo : str,
                       caminho_destino : str,
                       silencio : bool = False) -> bool:
  """
  Função responsável por baixar um arquivo do Drive por meio do seu ID e salvá-lo
  numa pasta no diretório de execução deste programa.

  ### Parâmetros:
  - ID_arquivo: String contendo o ID do arquivo do Drive a ser baixado.
  - caminho_destino: String contendo o caminho completo (contendo o nome e
  extensão do arquivo no final).
  - silencio: Bool responsável por imprimir no output ou não informações a respeito
  da execução deste processo.

  ### Retornos:
  - Bool referente ao sucesso (True) ou não (False) da execução total deste processo.
  """
  try:
    # URL no formato que o gdown utiliza para baixar arquivos do Google Drive por meio do ID
    url = f'https://drive.google.com/uc?export=download&id={ID_arquivo}'
    os.makedirs(os.path.dirname(caminho_destino),exist_ok=True)
    # Execução do processo de baixar o arquivo do Drive e trazê-lo para este ambiente de execução
    gdown.download(url, caminho_destino, quiet=silencio)
  except Exception as e:
    if not silencio:
      print(f'Erro na hora de baixar arquivo do Drive: {e.__class__.__name__}: {str(e)}')
    return False
  else:
    return True

def limparConsole():
    """
    Função responsável por limpar a tela de output/console.
    """
    global GOOGLE_COLAB
    global OS_ATUAL
    if GOOGLE_COLAB: # Se estivermos executando no Colab, melhor utilizar a função própria para limpar o output neste ambiente
        output.clear()
    elif OS_ATUAL.lower() == 'windows': # Se for Windows
        os.system('cls')
    else:   # Se for MAC/Linux
        os.system('clear')

def carregarPlanilha(caminho_planilha : str = ''):
  """
  Função responsável por carregar a planilha de metadados na memória RAM do programa.

  ### Parâmetros:
  - caminho_planilha: String contendo o caminho até o arquivo referente a
  planilha de metadados.

  ### Retornos:
  - None ou a instância da planilha carregada via openpyxl.
  """
  global CAMINHO_SKINNER
  if not caminho_planilha:
    caminho_planilha = os.path.join(CAMINHO_SKINNER,'planilha_metadados_woke.xlsx')
  try:
    # Carregar o workbook usando openpyxl
    wb = openpyxl.load_workbook(caminho_planilha)
  except Exception:
    return None
  else:
    return wb

# coletarInfoPlanilha(wb=wb,nome_colecao='HST',nome_trabalho='Trabalho 189')
def coletarInfoPlanilha(wb,
                        nome_colecao : str,
                        nome_trabalho : str) -> dict:
  """
  Função responsável por coletar e retornar as informações sobre Assuntos, Link
  da página no site do Repositório Institucional da UFSC e Link para o arquivo
  PDF do trabalho em questão.

  ### Parâmetros:
  - wb: Instância da planilha carregada via openpyxl.
  - nome_colecao: String contendo o nome da coleção onde se encontra o trabalho
  em questão.
  - nome_trabalho: String contendo o nome do Trabalho em questão.

  ### Retornos:
  - Dicionário contendo as informações do trabalho, caso o processo de coleta
  seja bem sucedido, caso contrário os valores das chaves do dicionário serão
  None.
  """
  global dic_nomes_alterados_abas_planilha
  dic_resultado = {'assuntos':None,'link_pagina':None,'link_pdf':None}
  try:
    nome_colecao_planilha = formatarNomeArquivoPlanilha(nome_colecao)
    if nome_colecao_planilha in dic_nomes_alterados_abas_planilha.keys():
      nome_colecao_planilha = dic_nomes_alterados_abas_planilha[nome_colecao_planilha]
    else:
      nome_colecao_planilha = nome_colecao_planilha[:31]
    ws=wb[nome_colecao_planilha]
    numero_trabalho = int(nome_trabalho.split()[-1])
    dic_resultado['assuntos'] = ws.cell(row=numero_trabalho+1, column=5).value
    dic_resultado['link_pagina'] = ws.cell(row=numero_trabalho+1, column=11).hyperlink.target
    dic_resultado['link_pdf'] = ws.cell(row=numero_trabalho+1, column=12).hyperlink.target
    return dic_resultado
  except Exception as e:
    print(e.__class__.__name__,str(e))
    return {'assuntos':None,'link_pagina':None,'link_pdf':None}


def salvarResultadosEmMsgPack(nome_variavel : str,
                              variavel_em_questao,
                              pasta_para_salvar : str) -> tuple[bool,str]:
    """
    Função responsável por salvar, no formato "msgpack", alguma variável deste
    ambiente numa pasta especificada.

    ### Parâmetros:
    - nome_variavel: String contendo o nome da variável que será dado ao arquivo.
    - variavel_em_questao: Variável propriamente dita que se deseja salvar num
    arquivo.
    - pasta_para_salvar: String contendo o caminho até a pasta onde se deseja
    salvar o arquivo da variável em questão.

    ### Retornos:
    - Tupla com dois elementos. O primeiro do tipo bool que faz referência ao
    status do processo (True bem sucedido, False falhou) e o segundo do tipo
    string contendo a mensagem referente ao status obtido.
    """
    try:
        # Se a pasta não existir, efetuamos sua criação
        if not os.path.exists(pasta_para_salvar):
            os.makedirs(pasta_para_salvar)

        # Obtendo os bytes da variável que queremos salvar
        variable_bytes = msgpack.packb(variavel_em_questao)

        # Se o nome do arquivo que salvará a variável não tiver o formato msgpack ao final, iremos adicioná-lo.
        if not nome_variavel.endswith('.msgpack'):
            nome_variavel += '.msgpack'

        # Salvando a variável desejada em bytes no formato .msgpack
        with open(os.path.join(pasta_para_salvar,nome_variavel),'wb') as f:
            f.write(variable_bytes)
            f.close()
            return True, f'Variável {nome_variavel} salva com sucesso no formato .msgpack'

    # Caso ocorra algum erro inesperado, trataremos o enviando como mensagem de falha juntamente com um status False (de falha/erro no processo)
    except Exception as e:
        error_message = f'{e.__class__.__name__}: {str(e)}'
        return False, error_message

def abrirArquivoMsgPack(full_filepath : str,
                        encoding_type : str = None):
    """
    Função responsável por abrir os arquivos no formato msgpack.

    ### Parâmetros:
    - full_filepath: String contendo o caminho completo até o arquivo que deseja-se
    abrir e extrair o conteúdo (variável salva).
    - encoding_type: String contendo o tipo de encoding, caso desejar.

    ### Retornos:
    - Variável salva (e agora aberta e lida) no arquivo msgpack.
    """
    if not full_filepath.endswith('.msgpack'):
        full_filepath += '.msgpack'
    if encoding_type:
        with open(full_filepath,'rb',encoding=encoding_type) as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded
    else:
        with open(full_filepath,'rb') as f:
            variable_bytes = f.read()
            variable_loaded = msgpack.unpackb(variable_bytes, raw=False)
            f.close()
            return variable_loaded


def extracaoTokensContexto(frase : list[str],
                           token : str,
                           i_token : int,
                           window : int,
                           dic_contexto : dict) -> dict:
  """
  Função responsável por atualizar, com base numa dada frase e no tamanho da janela
  de contexto, o dicionário que armazena as informações dos tokens de contexto para
  um determinado token central fornecido.

  ### Parâmetros:
  - frase: Lista de strings/tokens sendo uma frase tokenizada.
  - token: String contendo o token central/alvo, que está sendo analisado.
  - i_token: Índice do token na lista de tokens da frase atual.
  - window: Inteiro referente ao tamanho da janela de contexto.
  - dic_contexto: Dicionário contendo os tokens de contexto e suas ocorrências
  para o token alvo atual.

  ### Retornos:
  - Dicionário de contexto atualizado para o token central atual.
  """
  qtd_palavras = len(frase) # Coleta a quantidade de tokens presentes na frase fornecida para atualização de contexto
  for n in range(1,window+1): # A iteração será apenas no intervalo de 1 ao tamanho da janela de contexto
    indice_atual = i_token + n # Este laço for será responsável por coletar os tokens de contexto à direita do token central
    if indice_atual < qtd_palavras and indice_atual >= 0: # Se o índice atual na lista de tokens da frase for válido
      token_contexto = frase[indice_atual] # Armazenando o token de contexto analisado no momento
      if token_contexto != token: # Se o token de contexto for diferente do token central
        if token_contexto in dic_contexto.keys(): # Se o token de contexto encontrado já estiver dentro do dicionário de contexto do token central
          dic_contexto[token_contexto] += 1 # Adicionamos mais uma ocorrência
        else:
          dic_contexto[token_contexto] = 1 # Criamos uma chave no dicionário para o token de contexto e botamos uma ocorrência para ele

  # A mesma lógica do laço for de cima será aplicada neste laço for abaixo, porém contemplando os tokens de contexto à esquerda do token central/alvo
  for n in range(1,window+1):
    indice_atual = i_token - n  # Este laço for será responsável por coletar os tokens de contexto à direita do token central
    if indice_atual < qtd_palavras and indice_atual >= 0:
      token_contexto = frase[indice_atual]
      if token_contexto != token:
        if token_contexto in dic_contexto.keys():
          dic_contexto[token_contexto] += 1
        else:
          dic_contexto[token_contexto] = 1

  return dic_contexto # Após feitas as adições e alterações nos valores das ocorrências dentro do dicionário de contexto fornecido, retornamos ele atualizado

def formatarNomeArquivoPlanilha(nome_colecao : str) -> str:
  """
  Função responsável por formatar o nome da coleção encontrada nas pastas referentes
  ao corpus de textos pré-processados para o nome da coleção presente na planilha
  de metadados, o qual sofreu abreviações na tentativa de diminuir seu tamanho.

  ### Parâmetros:
  - nome_colecao: String contendo o nome da coleção referente a pasta da coleção
  no corpus de documentos pré-processados.

  ### Retornos:
  - String do nome da coleção fornecido com a formatação para bater com o nome da
  mesma coleção na planilha.
  """
  return nome_colecao.replace('Mestrado_Profissional','MP').replace('Universitaria','UNIV').replace('Desenvolvimento','DESENV').replace('Biotecnologia','BIOTEC').replace('Biologia','BIO').replace('Administracao','ADM').replace('Engenharia','ENG').replace('Historia','HST').replace('__','_').replace('_de_','_').replace('Programa_Pos_Graduacao','PG').replace('Propriedade','Prop').replace('Intelectual','Intelec').replace('Transferencia','Transf').replace('Tecnologia','Tec')


# Carregando planilha de metadados neste Programa

In [ ]:
if organizarAmbienteExecucao():
  # Link de compartilhamento do arquivo referente à Planilha de Metadados no Google Drive
  link_planilha_metadados_drive = 'https://docs.google.com/spreadsheets/d/1bXMirZN8NoyaXiRDj9CY19QDIuw9cPyB/edit?usp=drive_link&ouid=107024036721805330434&rtpof=true&sd=true'

  # Coletando o ID do arquivo
  id_planilha = coletar_ID_link_Dive(link_planilha_metadados_drive)

  if id_planilha:
    # Baixando arquivo da Planilha de Metadados neste ambiente
    status_download = baixarArquivoDrive(ID_arquivo=id_planilha,caminho_destino=CAMINHO_PLANILHA_METADADOS)
    limparConsole()
    if status_download:
      # Carregando planilha na RAM
      wb = carregarPlanilha()
      if wb:
        # Etapa adicional para lidar com nomes de abas iguais na Planilha de Metadados proveniente da limitação de caractéres (máximo de 31) no nome das abas
        dic_nomes_alterados_abas_planilha = {}
        lista_nomes_abas_planilha = []
        # Coletando os nomes das coleções via pastas do corpus de documentos pré-processados
        caminho_pasta_colecoes_pp = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'
        for nome_colecao in [formatarNomeArquivoPlanilha(pasta) for pasta in os.listdir(caminho_pasta_colecoes_pp) if '.' not in pasta]:
          nome_colecao_formatado = nome_colecao[:31] # Limitando o nome para 31 caractéres
          if nome_colecao_formatado in lista_nomes_abas_planilha: # Se o nome se repetir
            dic_nomes_alterados_abas_planilha[nome_colecao]=f'Página{len(dic_nomes_alterados_abas_planilha.keys())+1}' # O primeiro nome repetido terá seu nome na planilha como "Página1"
          else:
            lista_nomes_abas_planilha.append(nome_colecao_formatado)

        print('Sucesso ao carregar Planilha de Metadados.')

      else:
        print('Falha ao carregar planilha.')
    else:
      print('Falha no Download do arquivo da Planilha de Metadados.')
  else:
    print('Falha ao encontrar ID do arquivo referente Planilha de Metadados.')
else:
  print('Falha na organização do ambiente de execução.')

# Criação de arquivos

❗ *Executar apenas se a planilha for carregada adequadamente.*

- Foi removido do contexto a palavra idêntica à palavra central.
- Considerou que o contexto será construído apenas com os tokens que passarem no min_count.

In [ ]:
# Caminho até a pasta no Drive na qual será depositada os arquivos gerados por esse programa
caminho_skinner = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/SKINNER'

# Caminho até os corpus de treinos utilizados (HST, CFH, SAUDE-CORPO, UFSC)
caminho_corpus_treinos = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Treinamento do nosso modelo/Treinamento com temporalização'

# Lista contendo os nomes dos corpus de treinos ['HST-03-10', 'CFH-03-10', 'SAUDE-CORPO-03-10', 'Todas 2003 - 2006')
corpus_treinos = [corpus_utilizado for corpus_utilizado in os.listdir(caminho_corpus_treinos) if '.' not in corpus_utilizado and '(' not in corpus_utilizado]

for corpus_treino in corpus_treinos: # Iterando sobre os corpus para geração de arquivos referentes aos modelos treinados
  print('Corpus de treino atual:',corpus_treino)
  caminho_skinner_treino_atual = os.path.join(caminho_skinner,corpus_treino) # Caminho para salvar os arquivos gerados atualizado (caminho skinner + corpus de treino)
  os.makedirs(caminho_skinner_treino_atual,exist_ok=True) # Criando, caso não exista, a pasta para armazenar os arquivos dos modelos desse corpus de treino utilizado

  caminho_corpus_treino_inc = os.path.join(caminho_corpus_treinos,corpus_treino,'Com RE','Treinamento incremental') # Caminho até os modelos cuja as séries foram constrúidas de forma incremental (com atualização da rede)
  caminho_skinner_treino_atual_tipo = os.path.join(caminho_skinner_treino_atual,'Incremental') # Atualizando caminho para salvar arquivos
  os.makedirs(caminho_skinner_treino_atual_tipo,exist_ok=True)

  # Obtendo os modelos que melhor perfomaram nas analogias com seu modelo base e tiveram as séries temporais construídas
  caminhos_top_modelos_inc = sorted([os.path.join(caminho_corpus_treino_inc,top_modelo) for top_modelo in os.listdir(caminho_corpus_treino_inc) if top_modelo.startswith('Modelo')])
  print('Tipo de treino: Incremental')
  for n,caminho_top_modelo_inc in enumerate(caminhos_top_modelos_inc): # Iterando sobre os top modelos escolhidos para construção das séries temporais com base nos seus resultados nas analogias
    print('Modelo',n+1,'de',len(caminhos_top_modelos_inc))
    caminho_skinner_treino_atual_tipo_top_modelo = os.path.join(caminho_skinner_treino_atual,'Incremental',os.path.basename(caminho_top_modelo_inc)) # Atualizando caminho para salvar arquivos
    os.makedirs(caminho_skinner_treino_atual_tipo_top_modelo,exist_ok=True)

    # Obtendo os modelos referentes às séries temporais do Modelo X (top 3 ou top4)
    caminhos_modelos_temporais_inc = sorted([os.path.join(caminho_top_modelo_inc,modelo_temporal) for modelo_temporal in os.listdir(caminho_top_modelo_inc) if modelo_temporal.endswith('.model')])
    for s,caminho_modelo_temporal in enumerate(caminhos_modelos_temporais_inc): # Iterando sobre as séries temporais
      print('Série temporal:',s+1,'de',len(caminhos_modelos_temporais_inc))
      modelo = Word2Vec.load(caminho_modelo_temporal) # Carregando o modelo referente à série temporal atual
      nome_modelo = os.path.basename(caminho_modelo_temporal).replace('.model','') # Coletando seu nome
      window = modelo.window # Coletando a janela utilizada no treinamento
      vocab = modelo.wv.index_to_key # Coletando os tokens presentes no vocabulário
      print(nome_modelo)

      caminho_modelo_atual = os.path.join(caminho_skinner_treino_atual_tipo_top_modelo,nome_modelo) # Atualizando o caminho para salvar arquivos (aqui que será, de fato, upado)
      os.makedirs(caminho_modelo_atual,exist_ok=True)

      search = re.search(r'(\d{4})_(\d{4})',nome_modelo) # Procurando no nome do modelo da série temporal os dígitos de início e final de anos contemplados neste treinamento
      data_inicio = search.group(1) # Obtendo o ano inicial
      data_fim = search.group(2) # Obtendo o ano final
      print('Vocab:',len(vocab),'Data início:',data_inicio,'Data fim:',data_fim) # Printando informações do vocabulário e o intervalo de anos contemplado

      # Caminho até as pastas das coleções no corpus de documentos de textos pré-processados
      caminho_pasta_colecoes_pp = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'

      if corpus_treino == 'CFH-03-10': # Se o corpus de treino for do CFH, a lista de coleções contempladas será:
        lista_colecoes = ['Filosofia','Geografia','Geologia','Historia','Psicologia','Teses_e_dissertacoes_do_Centro_de_Filosofia_e_Ciencias_Humanas','Programa_de_Pos_Graduacao_Interdisciplinar_em_Ciencias_Humanas','Servico_Social','Sociologia_e_Ciencia_Politica','Sociologia_Politica','Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional','Ensino_de_Historia_Mestrado_Profissional']

      elif corpus_treino == 'HST-03-10': # Se o corpus de treino for do HST, a lista de coleções contempladas será:
        lista_colecoes = ['Historia']

      elif corpus_treino == 'SAUDE-CORPO-03-10': # Se o corpus de treino for do SAUDE-CORPO, a lista de coleções contempladas será:
        lista_colecoes = ['Biologia_Celular_e_do_Desenvolvimento','Biotecnologia_e_Biociencias','Ciencias_da_Reabilitacao','Ciencias_Medicas','Cuidados_Intensivos_e_Paliativos_Mestrado_Profissional',
                     'Educacao_Fisica','Enfermagem','Gestao_do_Cuidado_em_Enfermagem','Gestao_do_Cuidado_em_Enfermagem_Mestrado_Profissional','Medicina_Veterinaria_Convencional_e_Integrativa',
                     'Neurociencias','Saude_Coletiva','Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional','Saude_Publica','Programa_de_Pos_Graduacao_Multidisciplinar_em_Saude_Mestrado_Profissional']

       # Se o corpus de treino for do UFSC, a lista de coleções contempladas será:
      elif corpus_treino in ['Todas 2003 - 2006','UFSC 2003 - 2006', 'UFSC-03-06']: # Caso o modelo UFSC troque o nome da pasta de treino para algum destes outros
        lista_colecoes = [pasta for pasta in os.listdir(caminho_pasta_colecoes_pp) if '.' not in pasta]

      # Obtendo o caminho completo até as pastas das coleções no corpus pré-processado
      caminho_colecoes = sorted([os.path.join(caminho_pasta_colecoes_pp,pasta) for pasta in os.listdir(caminho_pasta_colecoes_pp) if '.' not in pasta in lista_colecoes])

      qtd_colecoes = len(caminho_colecoes) # Obtendo quantidade de coleções contempladas neste treinamento

      for c,caminho_colecao in enumerate(caminho_colecoes): # Iterando sobre cada coleção
        print('Coleção:',c+1,'de',qtd_colecoes)

        anos_neste_treinamento = [str(ano) for ano in range(int(data_inicio),int(data_fim)+1)] # Criando lista com os anos contemplados neste modelo

        # Obtendo caminho até os anos, dentro da pasta da coleção atual, que fizeram parte do treinamento do modelo da série temporal atual
        caminho_anos = sorted([os.path.join(caminho_colecao,ano) for ano in os.listdir(caminho_colecao) if ano.isdigit() and ano in anos_neste_treinamento])

        colecao = os.path.basename(caminho_colecao) # Obtendo o nome da coleção atual

        if not os.path.exists(os.path.join(caminho_modelo_atual,f'dic_{colecao}.msgpack')): # Se não tiver sido salvo um arquivo para esta coleção no diretório que era para salvá-lo, vamos então construí-lo.
          dic_info_colecao = {} # Inicialização do dicionário que irá armazenar as informações de contextos referentes à coleção atual

          for caminho_ano in caminho_anos: # Iterando sobre as pastas dos anos dessa coleção que participaram do treinamento do modelo da série temporal atual
            # Obtendo os trabalhos no ano atual
            trabalhos = sorted([trabalho for trabalho in os.listdir(caminho_ano) if trabalho.startswith('Trabalho')], key=lambda x: int(x.split()[1]))
            # Obtendo os caminhos completos até as pastas dos trabalhos do ano atual
            caminho_trabalhos = [os.path.join(caminho_ano,trabalho) for trabalho in trabalhos]

            for caminho_trabalho in caminho_trabalhos: # Iterando sobre as pastas dos trabalhos do ano atual
              caminho_arquivo_pp = os.path.join(caminho_trabalho,'pre_processamento_c_re.msgpack') # Caminho até o arquivo de texto pré-processado para o trabalho atual
              trabalho = os.path.basename(caminho_trabalho) # Obtendo nome do trabalho

              if os.path.exists(caminho_arquivo_pp): # Verificando se o arquivo de pré-processamento para este trabalho realmente existe
                # Criando chave no dicionário de contexto referente ao trabalho atual. Esta chave terá como valor um dicionário que conterá duas chaves: uma referente aos metadados que já será preenchida e outra aos tokens centrais/alvos que será preenchida ao decorrer da execução abaixo
                dic_info_colecao[trabalho] = {'metadados':coletarInfoPlanilha(wb=wb,nome_colecao=colecao,nome_trabalho=trabalho),
                                              'tokens_centrais':{}}
                try:
                  for frase in abrirArquivoMsgPack(caminho_arquivo_pp): # Iterando sobre cada frase presente no arquivo de texto pré-processado do trabalho atual
                    frase = [token for token in frase if token in vocab] # Filtrando a frase para apenas conter os tokens presentes no vocabulário do modelo ( https://stackoverflow.com/questions/50723303/how-is-word2vec-min-count-applied )
                    for indice_token,token in enumerate(frase): # Iterando sobre cada token presente na frase, identificando também seu índice
                      if token not in dic_info_colecao[trabalho]['tokens_centrais'].keys(): # Se o token central que está sendo analisado agora ainda não for uma chave para o dicionário de tokens_centrais do trabalho atual
                        dic_info_colecao[trabalho]['tokens_centrais'][token] = {} # Criaremos a chave com este token e atribuiremos o valor de um dicionário vazio (que será preenchido com os tokens de contexto e suas ocorrências nas frases em que estão dentro da janela de contexto para o token central atual)
                      # Atualiza-se o dicionário de contextos do trabalho atual, na chave referente ao contexto do token central atualmente analisado
                      dic_info_colecao[trabalho]['tokens_centrais'][token] = extracaoTokensContexto(frase=frase,token=token,i_token=indice_token,window=window,dic_contexto=dic_info_colecao[trabalho]['tokens_centrais'][token])

                except Exception as e:
                  print(e.__class__.__name__,str(e))
                  pass
                else:
                  for token in dic_info_colecao[trabalho]['tokens_centrais'].keys(): # Iterando sobre cada token central identificado no trabalho
                    # O dicionário de contexto do token central atual é ordenado para que os tokens de maior ocorrência fiquem em primeiro
                    dic_info_colecao[trabalho]['tokens_centrais'][token] = dict(sorted(dic_info_colecao[trabalho]['tokens_centrais'][token].items(), key=lambda item: item[1], reverse=True))

          if dic_info_colecao.keys():
            # Salva a variável referente ao dicionário de contextos da coleção atual nas pastas referentes ao treinamento da série temporal atual
            salvarResultadosEmMsgPack(nome_variavel=f'dic_{colecao}.msgpack',variavel_em_questao=dic_info_colecao,pasta_para_salvar=caminho_modelo_atual)
        else:
          print('Dados desta coleção já armazenados!')

  # Realiza-se o mesmo processo, mas agora contemplando os modelos em que as séries temporais foram construídas de maneira "temporal"
  caminho_corpus_treino_tmp = os.path.join(caminho_corpus_treinos,corpus_treino,'Com RE','Treinamento temporal')
  caminho_skinner_treino_atual_tipo = os.path.join(caminho_skinner_treino_atual,'Temporal')
  os.makedirs(caminho_skinner_treino_atual_tipo,exist_ok=True)

  caminhos_top_modelos_tmp = sorted([os.path.join(caminho_corpus_treino_tmp,top_modelo) for top_modelo in os.listdir(caminho_corpus_treino_tmp) if top_modelo.startswith('Modelo')])
  print('Tipo de treino: Temporal')
  for n,caminho_top_modelo_tmp in enumerate(caminhos_top_modelos_tmp):
    print('Modelo',n+1,'de',len(caminhos_top_modelos_tmp))
    caminho_skinner_treino_atual_tipo_top_modelo = os.path.join(caminho_skinner_treino_atual,'Temporal',os.path.basename(caminho_top_modelo_tmp))
    os.makedirs(caminho_skinner_treino_atual_tipo_top_modelo,exist_ok=True)

    caminhos_modelos_temporais_tmp = sorted([os.path.join(caminho_top_modelo_tmp,modelo_temporal) for modelo_temporal in os.listdir(caminho_top_modelo_tmp) if modelo_temporal.endswith('.model')])
    for s,caminho_modelo_temporal in enumerate(caminhos_modelos_temporais_tmp):
      print('Série temporal:',s+1,'de',len(caminhos_modelos_temporais_tmp))
      modelo = Word2Vec.load(caminho_modelo_temporal)
      nome_modelo = os.path.basename(caminho_modelo_temporal).replace('.model','')
      window = modelo.window
      vocab = modelo.wv.index_to_key
      print(nome_modelo)

      caminho_modelo_atual = os.path.join(caminho_skinner_treino_atual_tipo_top_modelo,nome_modelo)
      os.makedirs(caminho_modelo_atual,exist_ok=True)

      search = re.search(r'(\d{4})_(\d{4})',nome_modelo)
      data_inicio = search.group(1)
      data_fim = search.group(2)
      print('Vocab:',len(vocab),'Data início:',data_inicio,'Data fim:',data_fim)

      caminho_pasta_colecoes_pp = r'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Word Embeddings/Textos_pre_processados/Colecoes_textos_pre_processados'

      if corpus_treino == 'CFH-03-10':
        lista_colecoes = ['Filosofia','Geografia','Geologia','Historia','Psicologia','Teses_e_dissertacoes_do_Centro_de_Filosofia_e_Ciencias_Humanas','Programa_de_Pos_Graduacao_Interdisciplinar_em_Ciencias_Humanas','Servico_Social','Sociologia_e_Ciencia_Politica','Sociologia_Politica','Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional','Ensino_de_Historia_Mestrado_Profissional']

      elif corpus_treino == 'HST-03-10':
        lista_colecoes = ['Historia']

      elif corpus_treino == 'SAUDE-CORPO-03-10':
        lista_colecoes = ['Biologia_Celular_e_do_Desenvolvimento','Biotecnologia_e_Biociencias','Ciencias_da_Reabilitacao','Ciencias_Medicas','Cuidados_Intensivos_e_Paliativos_Mestrado_Profissional',
                     'Educacao_Fisica','Enfermagem','Gestao_do_Cuidado_em_Enfermagem','Gestao_do_Cuidado_em_Enfermagem_Mestrado_Profissional','Medicina_Veterinaria_Convencional_e_Integrativa',
                     'Neurociencias','Saude_Coletiva','Saude_Mental_e_Atencao_Psicossocial_Mestrado_Profissional','Saude_Publica','Programa_de_Pos_Graduacao_Multidisciplinar_em_Saude_Mestrado_Profissional']

      elif corpus_treino in ['Todas 2003 - 2006','UFSC 2003 - 2006','UFSC-03-06']: # Caso o modelo UFSC troque o nome da pasta de treino para algum destes outros
        lista_colecoes = [pasta for pasta in os.listdir(caminho_pasta_colecoes_pp) if '.' not in pasta]

      caminho_colecoes = sorted([os.path.join(caminho_pasta_colecoes_pp,pasta) for pasta in os.listdir(caminho_pasta_colecoes_pp) if '.' not in pasta in lista_colecoes])

      qtd_colecoes = len(caminho_colecoes)

      for c,caminho_colecao in enumerate(caminho_colecoes):
        print('Coleção:',c+1,'de',qtd_colecoes)

        anos_neste_treinamento = [str(ano) for ano in range(int(data_inicio),int(data_fim)+1)]

        caminho_anos = sorted([os.path.join(caminho_colecao,ano) for ano in os.listdir(caminho_colecao) if ano.isdigit() and ano in anos_neste_treinamento])
        colecao = os.path.basename(caminho_colecao)

        if not os.path.exists(os.path.join(caminho_modelo_atual,f'dic_{colecao}.msgpack')):
          dic_info_colecao = {}

          for caminho_ano in caminho_anos:
            trabalhos = sorted([trabalho for trabalho in os.listdir(caminho_ano) if trabalho.startswith('Trabalho')], key=lambda x: int(x.split()[1]))
            caminho_trabalhos = [os.path.join(caminho_ano,trabalho) for trabalho in trabalhos]

            for caminho_trabalho in caminho_trabalhos:
              caminho_arquivo_pp = os.path.join(caminho_trabalho,'pre_processamento_c_re.msgpack')
              trabalho = os.path.basename(caminho_trabalho)

              if os.path.exists(caminho_arquivo_pp):
                dic_info_colecao[trabalho] = {'metadados':coletarInfoPlanilha(wb=wb,nome_colecao=colecao,nome_trabalho=trabalho),
                                              'tokens_centrais':{}}
                try:
                  for frase in abrirArquivoMsgPack(caminho_arquivo_pp):
                    frase = [token for token in frase if token in vocab]
                    for indice_token,token in enumerate(frase):
                      if token not in dic_info_colecao[trabalho]['tokens_centrais'].keys():
                        dic_info_colecao[trabalho]['tokens_centrais'][token] = {}
                      dic_info_colecao[trabalho]['tokens_centrais'][token] = extracaoTokensContexto(frase=frase,token=token,i_token=indice_token,window=window,dic_contexto=dic_info_colecao[trabalho]['tokens_centrais'][token])

                except Exception as e:
                  print(e.__class__.__name__,str(e))
                  pass
                else:
                  for token in dic_info_colecao[trabalho]['tokens_centrais'].keys():
                    dic_info_colecao[trabalho]['tokens_centrais'][token] = dict(sorted(dic_info_colecao[trabalho]['tokens_centrais'][token].items(), key=lambda item: item[1], reverse=True))

          if dic_info_colecao.keys():
            salvarResultadosEmMsgPack(nome_variavel=f'dic_{colecao}.msgpack',variavel_em_questao=dic_info_colecao,pasta_para_salvar=caminho_modelo_atual)
        else:
          print('Dados desta coleção já armazenados!')